In [1]:
from map_engine.map_generator import map
from encounter_engine.world_encounters import *
import pickle
from matplotlib import colors
from quest_engine.quest_generator import *
import matplotlib.pyplot as plt

from quest_engine.freytags_fitness import freytags

In [2]:
#nw = map()
#nw.populate_map(n_locations=1024, name="terrain", relaxed=True, k=100)
#nw.populate_map(n_locations=4096, name="civilisation", relaxed=True, k=250)
#nw.populate_map(n_locations=1024, name="story_act", relaxed=True, k=250)
#nw.populate_map(n_locations=1024, name="threat", relaxed=True, k=100)
#
#nw.attribute_view(seed_1=24, seed_2=34,
#                  map_name="terrain", view_name="terrain")
#nw.attribute_view(seed_1=98, seed_2=50, map_name="civilisation",
#                  view_name="civilisation", double=False)
#nw.attribute_view(seed_1=34, seed_2=34, map_name="threat",
#                  view_name="threat", double=False)
#nw.attribute_view(seed_1=16, seed_2=16, map_name="story_act",
#                  view_name="story_act", double=False)
#
#nw.attribute_centroids("civilisation")
#nw.attribute_centroids("terrain")
#nw.land_mask()
#
#with open('map_nw', 'wb') as f:
#    pickle.dump(nw, f)


with open("map_nw", "rb") as f:
    nw = pickle.load(f)

In [ ]:
qs = quest_pop(nw,act=2, pop_size=200, steps=3)
q = qs.evolve(gens=1,early_stop=True, mu_p=0.40, xo="pmx", mutation="random_point",print_it=True)


In [ ]:
ql = np.select(qs.population)

In [ ]:
freytags([q.difficulty for q in ql])

In [ ]:
[q.difficulty for q in ql]

In [ ]:
#q = sorted(qs.population, key=operator.attrgetter('fitness'))[-1]
print("Total fitness" ,q.fitness)
print("Quest fitnesses",q.fitnesses)
print("Quest path",q.path)
print("Quest act",q.path_act)
print("Quest threat",q.check_threat())
print("Quest Freytags", q.freytags_fitnesses)
print("Quest ", q.freytags)

In [ ]:

lst1, lst2 = zip(*sorted(qs.population, key=operator.attrgetter('fitness'))[-1].path)

In [ ]:
lst1, lst2 = zip(*a)

In [ ]:

cmap = colors.ListedColormap(
    ["blue", "white", "yellowgreen", "khaki", "lawngreen", "slategrey", "darkgreen"])
cmap_arcs = colors.ListedColormap(
    ["blue", "khaki", "khaki", "khaki", "khaki", "khaki", "khaki", "brown", "brown", "brown", "black", ])

ter = nw.views["terrain"].copy()
civ = nw.views["civilisation"].copy()
thr = nw.views["threat"].copy()
arc = nw.views["story_act"].copy()

fig, ax = plt.subplots(2, 2)
fig.set_dpi(150)
fig.set_size_inches(24, 24)

ax[0, 1].imshow(ter.T, cmap=cmap, alpha=.66)
ax[0, 1].set_title("Terrain")
ax[0, 1].scatter(lst1, lst2, color="black", s=20)

#ax[0, 1].scatter([i[0] for i in q.path], [i[1] for i in q.path], color="black", s=15)
#ax[0, 1].plot([i[0] for i in q.path], [i[1] for i in q.path], color="red",linestyle="-.",linewidth=1, alpha=.5)

ax[1, 1].imshow(civ.T, cmap=cmap_arcs, alpha=0.66)
ax[1, 1].set_title("Civilisation")
ax[1, 1].scatter(lst1, lst2, color="black", s=20)

#ax[1, 1].scatter([i[0] for i in q.path], [i[1] for i in q.path], color="black", s=15)
#ax[1, 1].plot([i[0] for i in q.path], [i[1] for i in q.path], color="red",linestyle="-.",linewidth=1, alpha=.5)

ax[1, 0].imshow(thr.T, cmap="Reds", alpha=0.66)
ax[1, 0].set_title("Threat")
ax[1, 0].scatter(lst1, lst2, color="black", s=20)

#ax[1, 0].scatter([i[0] for i in q.path], [i[1] for i in q.path], color="black", s=15)
#ax[1, 0].plot([i[0] for i in q.path], [i[1] for i in q.path], color="red",linestyle="-.",linewidth=1, alpha=.5)

ax[0, 0].imshow(arc.T, cmap="Blues", alpha=0.66)
ax[0, 0].set_title("Story Act")
ax[0, 0].scatter(lst1, lst2, color="black", s=20)

#ax[0, 0].scatter([i[0] for i in q.path], [i[1] for i in q.path], color="black", s=15)
#ax[0, 0].plot([i[0] for i in q.path], [i[1] for i in q.path], color="red",linestyle="-.",linewidth=1, alpha=.5)


In [ ]:
fig.savefig("example_world_encounter.png")

In [ ]:
sorted([1,2,3,4,566,7,21])[-3:]

In [ ]:
params = {
    "gens": 100,
    "pop_size": 200,
    "mu_p": .5,
    "xo": "pmx",
    "mutation": "random_point",
    "print_it": True
}

ql = quest_library(nw,shelf_size=100,params=params)

In [ ]:

#with open("quest_library", "wb") as f:
#    ql = pickle.dump(ql,f)


with open("quest_library", "rb") as f:
    ql = pickle.load(f)

In [7]:
a=ql.curate(5,max_act=2)

Generation:  0
Fitness:  -2000
Generation:  1
Fitness:  -3000
Generation:  2
Fitness:  -3000
Generation:  3
Fitness:  -3000
Generation:  4
Fitness:  -1000
Generation:  5
Fitness:  -2000
Generation:  6
Fitness:  -2000
Generation:  7
Fitness:  -2000
Generation:  8
Fitness:  -2000
Generation:  9
Fitness:  -2000
Generation:  10
Fitness:  -2000
Generation:  11
Fitness:  -2000
Generation:  12
Fitness:  -2000
Generation:  13
Fitness:  -2000
Generation:  14
Fitness:  -2000
Generation:  15
Fitness:  -2000
Generation:  16
Fitness:  -2000
Generation:  17
Fitness:  -2000
Generation:  18
Fitness:  -2000
Generation:  19
Fitness:  -2000
Generation:  20
Fitness:  -2000
Generation:  21
Fitness:  -2000
Generation:  22
Fitness:  -2000
Generation:  23
Fitness:  -2000
Generation:  24
Fitness:  -2000
Generation:  25
Fitness:  -2000
Generation:  26
Fitness:  -2000
Generation:  27
Fitness:  -2000
Generation:  28
Fitness:  -2000
Generation:  29
Fitness:  -2000
Generation:  30
Fitness:  -2000
Generation:  31
Fi

KeyboardInterrupt: 

In [ ]:
curate_fitness(a)

In [ ]:
curate_xo(a, b)